In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline


In [2]:
# lieux_all = pd.read_csv("lieux.csv")
# vehicules_all = pd.read_csv("vehi.csv")
caracteristiques_all = pd.read_csv("../raw_data/carac.csv")


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_50410/149342845.py:3: DtypeWarning: Columns (4,10,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristiques_all = pd.read_csv("../raw_data/carac.csv")


In [3]:
caracteristiques_all.isna().sum()


Num_Acc          55302
an                   0
mois                 0
jour                 0
hrmn                 0
lum                  0
agg                  0
int                  0
atm                 73
col                 19
com                  2
adr             144509
gps             696821
lat             487068
long            487072
dep                  0
Accident_Id    1121571
dtype: int64

# Chargement des données CSV

In [4]:
lieux_df = pd.read_csv('../clean_data/lieux_clean.csv')
usagers_df = pd.read_csv('../clean_data/usagers_clean.csv')
vehicules_df = pd.read_csv('../clean_data/vehicules_clean.csv')
caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv')


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_50410/4187407229.py:4: DtypeWarning: Columns (6,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv')


In [5]:
caracteristiques_df.isna().sum()

caracteristiques_df[caracteristiques_df['Num_Acc'].isna()]


,Num_Acc,lum,agg,int,atm,col,com,adr,lat,long,dep,date
116623,NaN,1,2,3,1.0,3.0,26198,TEIL(vieille route du),"44,5594200000","4,7257200000",26,NaN
116624,NaN,1,2,3,1.0,3.0,25204,Miranda,"46,9258100000","6,3462000000",25,NaN
116625,NaN,1,2,6,1.0,2.0,22360,ROND POINT DE BREZILLET,"48,4931620000","-2,7604390000",22,NaN
116626,NaN,1,2,3,8.0,6.0,16102,LOHMEYER (RUE),"45,6926520000","-0,3262900000",16,NaN
116627,NaN,1,1,1,1.0,2.0,13103,ROUTE DE JEAN MOULIN-RN 538,"43,6755790366","5,0927031775",13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
171920,NaN,3,1,1,1.0,6.0,2B293,D71,"42,3101650000","9,4785830000",2B,NaN
171921,NaN,3,1,1,1.0,6.0,84074,D973,"43,7531640000","5,2244760000",84,NaN
171922,NaN,3,1,1,9.0,6.0,74001,D22,"46,2825320000","6,7328060000",74,NaN
171923,NaN,1,1,3,1.0,3.0,81099,Chemin Toulze,"43,9272650000","1,9156370000",81,NaN


In [6]:
# fonction rolliwn window ? applique une fonction sur une partie précise


# Merge de la données


In [7]:
# Création d'une colonne uniques pour merger les df usagers et véhicules

usagers_df['aug'] = usagers_df['Num_Acc'].astype(str) + usagers_df['num_veh'].astype(str)
vehicules_df['aug'] = vehicules_df['Num_Acc'].astype(str) + vehicules_df['num_veh'].astype(str)


KeyError: 'num_veh'

In [ ]:
vehicules_df.shape
vehicules_df = vehicules_df[vehicules_df['catv']==1]


In [ ]:
# On drop les duplicates lorsqu'il y a une plusieurs usagers dans un même véhicules et pour le même accident
# On crée une colonne pour y ajouter le nombre d'usagers par véhicules


usagers_df['dup_count'] = usagers_df.groupby('aug')['aug'].transform('count')
usagers_df =usagers_df.drop_duplicates(subset=['aug'], keep='first')


In [ ]:
print(type(vehicules_df))
print(type(usagers_df))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [13]:
# On merge véhicules et usagers puis on ne garde que les accidents impliquant un vélo
# On clean et on drop les doublons

vehi_usa = pd.merge(vehicules_df, usagers_df, on='aug', how='inner')
vehi_usa = vehi_usa.drop(columns=['num_veh_x', 'num_veh_y', 'Num_Acc_y'])
vehi_usa = vehi_usa.rename(columns={'Num_Acc_x': 'Num_Acc'})


KeyError: 'aug'

In [12]:
vehi_usa['dup_count'].value_counts()


NameError: name 'vehi_usa' is not defined

In [11]:
print(vehi_usa['Num_Acc'].duplicated().any())
print(caracteristiques_df['Num_Acc'].duplicated().any())


NameError: name 'vehi_usa' is not defined

In [10]:
duplicates_velo = vehi_usa[vehi_usa['Num_Acc'].duplicated(keep=False)]
duplicates_velo.info()


NameError: name 'vehi_usa' is not defined

In [15]:
# On merge les df caractéristiques et lieux

velo_df = pd.merge(vehi_usa, lieux_df, on='Num_Acc', how='left')
velo_df = pd.merge(velo_df, caracteristiques_df, on='Num_Acc', how='left')


In [16]:
velo_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 88521 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    88521 non-null  int64  
 1   catv       88521 non-null  int64  
 2   obs        88521 non-null  float64
 3   obsm       88521 non-null  float64
 4   choc       88521 non-null  float64
 5   manv       88521 non-null  float64
 6   aug        88521 non-null  object 
 7   grav       88521 non-null  int64  
 8   sexe       88521 non-null  int64  
 9   trajet     88521 non-null  int64  
 10  secu       88521 non-null  float64
 11  locp       88521 non-null  float64
 12  actp       88521 non-null  int64  
 13  etatp      88521 non-null  float64
 14  an_nais    88521 non-null  float64
 15  dup_count  88521 non-null  int64  
 16  catr       88521 non-null  float64
 17  circ       88521 non-null  float64
 18  nbv        88521 non-null  float64
 19  vosp       88521 non-null  float64
 20  prof  

In [17]:
# On remplace les valeurs manquantes par la valeur la plus fréquente
velo_df['lum'].fillna(velo_df['lum'].mode()[0], inplace=True)
velo_df['agg'].fillna(velo_df['agg'].mode()[0], inplace=True)
velo_df['atm'].fillna(velo_df['atm'].mode()[0], inplace=True)
velo_df['atm'] = velo_df['atm'].replace({ -1: 1, 9: 1})


In [18]:
# Obtenez la distribution des valeurs
distribution = velo_df['int'].value_counts(normalize=True)
# Obtenez les valeurs manquantes
missing = velo_df['int'].isnull()
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'int'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)


In [19]:
# Obtenez la distribution des valeurs
distribution = velo_df['col'].value_counts(normalize=True)
# Obtenez les valeurs manquantes
missing = velo_df['col'].isnull()
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'col'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)
# Je remplace le -1 par 1
velo_df['col'] = velo_df['col'].replace({ -1: 3})


In [20]:
velo_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 88521 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    88521 non-null  int64  
 1   catv       88521 non-null  int64  
 2   obs        88521 non-null  float64
 3   obsm       88521 non-null  float64
 4   choc       88521 non-null  float64
 5   manv       88521 non-null  float64
 6   aug        88521 non-null  object 
 7   grav       88521 non-null  int64  
 8   sexe       88521 non-null  int64  
 9   trajet     88521 non-null  int64  
 10  secu       88521 non-null  float64
 11  locp       88521 non-null  float64
 12  actp       88521 non-null  int64  
 13  etatp      88521 non-null  float64
 14  an_nais    88521 non-null  float64
 15  dup_count  88521 non-null  int64  
 16  catr       88521 non-null  float64
 17  circ       88521 non-null  float64
 18  nbv        88521 non-null  float64
 19  vosp       88521 non-null  float64
 20  prof  

In [21]:
velo_df = velo_df[~(velo_df.adr.isna() & velo_df.lat.isna() & velo_df.long.isna() & velo_df.com.isna() & velo_df.dep.isna())]


In [22]:
velo_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 83233 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    83233 non-null  int64  
 1   catv       83233 non-null  int64  
 2   obs        83233 non-null  float64
 3   obsm       83233 non-null  float64
 4   choc       83233 non-null  float64
 5   manv       83233 non-null  float64
 6   aug        83233 non-null  object 
 7   grav       83233 non-null  int64  
 8   sexe       83233 non-null  int64  
 9   trajet     83233 non-null  int64  
 10  secu       83233 non-null  float64
 11  locp       83233 non-null  float64
 12  actp       83233 non-null  int64  
 13  etatp      83233 non-null  float64
 14  an_nais    83233 non-null  float64
 15  dup_count  83233 non-null  int64  
 16  catr       83233 non-null  float64
 17  circ       83233 non-null  float64
 18  nbv        83233 non-null  float64
 19  vosp       83233 non-null  float64
 20  prof  

In [23]:
# Réinitialisez l'index de votre DataFrame
velo_df = velo_df.reset_index(drop=True)

# Obtenez la distribution des valeurs
distribution = velo_df['date'].value_counts(normalize=True)

# Obtenez les valeurs manquantes
missing = velo_df['date'].isnull()

# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'date'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)


In [ ]:
velo_df.drop(columns=['catv'], inplace=True)


In [24]:
velo_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83233 entries, 0 to 83232
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    83233 non-null  int64  
 1   catv       83233 non-null  int64  
 2   obs        83233 non-null  float64
 3   obsm       83233 non-null  float64
 4   choc       83233 non-null  float64
 5   manv       83233 non-null  float64
 6   aug        83233 non-null  object 
 7   grav       83233 non-null  int64  
 8   sexe       83233 non-null  int64  
 9   trajet     83233 non-null  int64  
 10  secu       83233 non-null  float64
 11  locp       83233 non-null  float64
 12  actp       83233 non-null  int64  
 13  etatp      83233 non-null  float64
 14  an_nais    83233 non-null  float64
 15  dup_count  83233 non-null  int64  
 16  catr       83233 non-null  float64
 17  circ       83233 non-null  float64
 18  nbv        83233 non-null  float64
 19  vosp       83233 non-null  float64
 20  prof  

In [27]:
print(velo_df['com'])


0         52
1        586
2        400
3         42
4        252
        ... 
83228    611
83229    611
83230    611
83231    611
83232    611
Name: com, Length: 83233, dtype: object


In [25]:
velo_df.to_csv('../clean_data/velo_df.csv', index=False)


# preprocess

In [38]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, Col
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import datetime as dt


In [39]:
def preprocess_features(X: pd.DataFrame) -> np.ndarray:

    def create_sklearn_preprocessor() -> ColumnTransformer:

        ohe_categories = ['obs','obsm','choc','manv','trajet','locp','atp','etatp','catr','circ','vosp','prof','plan','surf','infra','situ','lum','int','col']

        # Création du pipeline ohe

        ohe_pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    ('ohe', OneHotEncoder(categories='auto', sparse_output=False, handle_unknown='ignore'), ohe_categories)
                ],
                remainder='passthrough'
            )
        )

        # Création du pipeline grav
        grav_pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    ('grav', FunctionTransformer(lambda x: x.map({1: 'Indemne', 2: 'Blessé léger',
                                                                3: 'Blessé hospitalisé', 4: 'Tué'}),
                                                validate=False), ['grav'])
                ],
                remainder='passthrough'
            )
        )



        def calculate_age(df):
            # Extrait les 4 premiers chiffres de 'Num_Acc' comme année de l'accident
            df['annee_accident'] = df['Num_Acc'].astype(str).str[:4]

            # Convertit 'annee_accident' en type entier
            df['annee_accident'] = df['annee_accident'].astype(int)

            # Calcul de l'âge à la date de l'accident
            df['age_at_accident'] = df['annee_accident'] - df['an_nais']

            # Supprime les colonnes intermédiaires si nécessaire
            df = df.drop(['annee_accident'], axis=1)

            return df

        # Création du pipeline
        age_pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    ('age', FunctionTransformer(calculate_age, validate=False), ['Num_Acc', 'an_nais'])
                ],
                remainder='passthrough'
            )
        )


        final_pipeline = make_pipeline(
            ohe_pipe,
            grav_pipe,
            age_pipe,
            remainder='passthrough',
        # Garde les colonnes non utilisées
        )

        return final_pipeline

    preprocessor = create_sklearn_preprocessor()
    X_processed = preprocessor.fit_transform(X)

    print("✅ X_processed, with shape", X_processed.shape)

    return X_processed


,Num_Acc,catv,obs,obsm,choc,manv,aug,grav,sexe,trajet,...,int,atm,col,com,adr,lat,long,dep,date,age_at_accident
0,201800000004,1,0.0,2.0,8.0,1.0,201800000004B01,3,1,5,...,1.0,7.0,3.0,52,30 rue Jules Guesde,5051974.0,289123.0,590,2018-05-05 17:35:00,5.0
1,201800000020,1,12.0,0.0,7.0,1.0,201800000020B01,3,1,9,...,1.0,1.0,7.0,586,rue de wachemy,5054129.0,318902.0,590,2018-05-27 16:00:00,21.0
2,201800000044,1,0.0,0.0,8.0,2.0,201800000044A01,3,1,5,...,1.0,7.0,3.0,400,rue de maroeuil,5062556.0,262903.0,590,2018-09-29 18:00:00,71.0
3,201800000051,1,0.0,0.0,4.0,2.0,201800000051B01,2,1,5,...,1.0,2.0,2.0,42,Rte départementale,5053908.0,326460.0,590,2018-08-26 15:30:00,78.0
4,201800000056,1,0.0,2.0,6.0,2.0,201800000056B01,3,1,5,...,1.0,1.0,2.0,252,105 rue de quesnoy sur d,5068934.0,296942.0,590,2018-04-10 15:20:00,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83228,201500058611,1,14.0,2.0,1.0,17.0,201500058611B01,4,1,5,...,1.0,1.0,3.0,611,mamoudzou centre-Préfect,0.0,0.0,976,2015-11-05 16:50:00,55.0
83229,201500058616,1,0.0,2.0,1.0,1.0,201500058616B01,3,1,1,...,6.0,1.0,3.0,611,doujani-Mosquée doujani,0.0,0.0,976,2015-09-28 16:50:00,51.0
83230,201500058626,1,0.0,1.0,1.0,2.0,201500058626A01,1,1,9,...,1.0,1.0,6.0,611,m'tsapere-Dahilou (rue),0.0,0.0,976,2015-10-07 14:53:00,13.0
83231,201500058629,1,1.0,0.0,1.0,15.0,201500058629A01,4,1,5,...,4.0,1.0,3.0,611,mamoudzou centre-Préfect,0.0,0.0,976,2015-10-13 11:35:00,13.0


In [29]:
df_carac.columns
{k: 'string' for k in df_carac.columns}


{'Num_Acc': 'string',
 'lum': 'string',
 'agg': 'string',
 'int': 'string',
 'atm': 'string',
 'col': 'string',
 'com': 'string',
 'adr': 'string',
 'lat': 'string',
 'long': 'string',
 'dep': 'string',
 'date': 'string'}

In [31]:
dtype = {'Num_Acc': 'string',
 'lum': 'string',
 'agg': 'string',
 'int': 'string',
 'atm': 'string',
 'col': 'string',
 'com': 'string',
 'adr': 'string',
 'lat': 'string',
 'long': 'string',
 'dep': 'string',
 'date': 'string'}


In [32]:
df_carac = pd.read_csv('../clean_data/caracteristiques_clean.csv', dtype=dtype)


In [34]:
df_carac.isna().sum()


Num_Acc     55302
lum             0
agg             0
int             0
atm            73
col            19
com             2
adr        144509
lat        487068
long       487072
dep             0
date       218404
dtype: int64

In [12]:
df_carac.lat.value_counts()


0.0              101444
4.0               22481
0.0               16395
5.0                 406
4800000.0            96
                  ...  
44,5487800000         1
48,7739030000         1
49,0298282152         1
48,3988880000         1
2129833.0             1
Name: lat, Length: 398487, dtype: int64

In [23]:
import pandas as pd

def reformatter_lat_lon(dataframe):
    """
    Reformate les colonnes de latitude et de longitude d'une DataFrame Pandas,
    en remplaçant les virgules par des points et en assurant que les valeurs sont dans un format décimal correct.
    """

    if 'lat' in dataframe.columns and 'long' in dataframe.columns:
        for col in ['lat', 'long']:
            dataframe[col] = pd.to_numeric(dataframe[col].astype(str).str.replace(',', '.'), errors='ignore')
    else:
        raise ValueError("La DataFrame doit contenir les colonnes 'lat' et 'long'")

    return dataframe


In [53]:
def lat_(x):
    if 41 <= int(x[:2]) <= 52: return True
    return False


def long_(x):
    if -8 <= int(x[:2]) <= 10: return True
    return False

long_('2,0')


ValueError: invalid literal for int() with base 10: '2,'

In [ ]:
mini lat = 41
maxi lat = 51
mini long = -8
maxi long = 10


In [38]:
df_carac.long.str.len().value_counts().sort_index()


1      33498
3     107378
4          9
5         93
6       2190
7      21597
8     263512
9     128581
10      1898
11     16387
12     92036
13      2215
14     17001
15      3079
16       327
Name: long, dtype: Int64

In [54]:
def test_lat(x):
    if 41 <= int(x[:2]) <= 52: return True
    return False



test_lat('4476,4')


True

In [41]:
df_carac[df_carac.long.str.len()==15]


,Num_Acc,lum,agg,int,atm,col,com,adr,lat,long,dep,date
116689,<NA>,5,2,1,1.0,6.0,97225,Rue Bouille,"14,7437370000","-61,1762680000",972,<NA>
116724,<NA>,3,1,1,1.0,2.0,97307,RN1 VOIE RAPIDE,"4,8953090000","-52,3298120000",973,<NA>
116831,<NA>,1,1,1,2.0,3.0,97218,N3,"14,7428450000","-61,0986190000",972,<NA>
116859,<NA>,3,1,1,1.0,6.0,97103,La Jaille,"16,2521140000","-61,5840770000",971,<NA>
116861,<NA>,3,1,1,1.0,1.0,97107,lieu dit bananier - RN1,"15,9968100000","-61,6070800000",971,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
302804,202100056471.0,1,2,3,1.0,3.0,97128,Boulevard Hegesipe Ibene,"16,2245900000","-61,3794830000",971,<NA>
302805,202100056472.0,3,1,1,1.0,6.0,97102,Guéry,"16,4513060000","-61,4783390000",971,<NA>
302806,202100056473.0,1,2,1,1.0,6.0,97118,Chemin de Morne Bourg,"16,1865690000","-61,5987220000",971,<NA>
302807,202100056474.0,5,2,1,1.0,1.0,97701,"rt209, face aéroport, st jean","17,9033180000","-62,8443340000",977,<NA>


In [ ]:
## Passer dtype sur les read_csv
